In [ ]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init stereo aruco detector scene builder

In [ ]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

In [ ]:
from pkg.planning.filtering.reach_filter import ReachTrainer
from pkg.controller.combined_robot import *

In [ ]:
rtrain = ReachTrainer(scene_builder=s_builder)

In [ ]:
def fn_rpy(radius,theta, height, azimuth_loc, zenith, ee_dist, rot_z):
    R = np.matmul(hori2mat(theta=0, azimuth_loc=azimuth_loc, zenith=zenith), Rot_axis(3, rot_z))
    r,p,y = Rot2rpy(R)
    return (radius, theta, height, r,p,y, radius**2, ee_dist, ee_dist**2)

def fn_quat(radius,theta, height, azimuth_loc, zenith, ee_dist, rot_z):
    R = np.matmul(hori2mat(theta=0, azimuth_loc=azimuth_loc, zenith=zenith), Rot_axis(3, rot_z))
    x,y,z,w = Rotation.from_dcm(R).as_quat()
    return (radius, theta, height, x,y,z,w, radius**2, ee_dist, ee_dist**2)

def fn_rmat(radius,theta, height, azimuth_loc, zenith, ee_dist, rot_z):
    R = np.matmul(hori2mat(theta=0, azimuth_loc=azimuth_loc, zenith=zenith), Rot_axis(3, rot_z))
    r11, r12, r13, r21, r22, r23, r31, r32, r33 = R.flatten()
    return (radius, theta, height, r11, r12, r13, r21, r22, r23, r31, r32, r33, radius**2, ee_dist, ee_dist**2)

def fn_zxvec(radius,theta, height, azimuth_loc, zenith, ee_dist, rot_z):
    R = np.matmul(hori2mat(theta=0, azimuth_loc=azimuth_loc, zenith=zenith), Rot_axis(3, rot_z))
    z1, z2, z3 = R[:,2]
    x1, x2, x3 = R[:,0]
    return (radius, theta, height, z1, z2, z3, x1, x2, x3, radius**2, ee_dist, ee_dist**2)

def fn_pure(radius,theta, height, azimuth_loc, zenith, ee_dist, rot_z):
    return (radius, theta, height, azimuth_loc, zenith, ee_dist, rot_z)

def fn_re_sq(radius,theta, height, azimuth_loc, zenith, ee_dist, rot_z):
    return (radius, theta, height, azimuth_loc, zenith, radius**2, ee_dist, ee_dist**2, rot_z)

fn_list = [fn_rpy, fn_quat, fn_rmat, fn_zxvec, fn_pure, fn_re_sq]
for fn in fn_list:
    fn(*rtrain.samplevec_list_test[0])
print("function all sane")

In [ ]:
C_svm_list = [1000, 2000, 4000, 8000]
test_panda_set_dict = {}
for fn in fn_list:
    test_panda_set = {}
    for C_svm in C_svm_list:
        test_panda = rtrain.load_and_learn(RobotType.panda, C_svm=C_svm, feature_fn=fn)
        test_panda_set[str(C_svm)] = test_panda
    test_panda_set_dict[fn.__name__] = test_panda_set

In [ ]:
from pkg.utils.plot_utils import *
fn_names = map(lambda x:x.__name__, fn_list)
fn_maxvals = [max(test_panda_set_dict[fname].values()) for fname in fn_names]
plt.figure(figsize=(15,5))
plt.subplot(1,3,(1,2))
grouped_bar(test_panda_set_dict, groups=fn_names, options=map(str,C_svm_list))
plt.subplot(1,3,3)
plt.bar(fn_names, fn_maxvals)
plt.grid()
maxval = np.max(fn_maxvals)
idc_max = np.where(np.array(fn_maxvals)==maxval)[0]
print("fn:   "+"".join(["{:>10}".format(fname) for fname in fn_names]))
print("vals: "+"".join(["{:10}".format(val) for val in fn_maxvals]))
print("max: {} - {}".format(maxval, idc_max))

In [ ]:
import matplotlib.pyplot as plt
# plt.plot(C_svm_list, test_indy_list)
plt.plot(C_svm_list, test_panda_list)
plt.grid()

## load and test

In [ ]:
rtrain.load_and_test(RobotType.indy7gripper)

In [ ]:
rtrain.load_and_test(RobotType.panda)

## Update label

In [ ]:
from pkg.planning.filtering.reach_filter import *

In [ ]:
ROBOT_TYPE = RobotType.indy7
TIP_LINK = "indy0_tcp"
TIMEOUT = 0.3
DATA_DIVS = ["train", "test"]
UPDATE_LABELS = [True] # to reduce feasible ones, as joint limit is reduced

In [ ]:
time_lists = []
for DATA_DIV in DATA_DIVS:
    rtrain.update_label(robot_type=ROBOT_TYPE, tip_link=TIP_LINK, data_div=DATA_DIV, update_labels=UPDATE_LABELS, timeout=TIMEOUT)
    time_lists.append(deepcopy(rtrain.time_list))
import matplotlib.pyplot as plt
plt.subplot(1,2,1)
plt.plot(sorted(time_lists[0]), '.')
plt.subplot(1,2,2)
plt.plot(sorted(time_lists[0]), '.')

## Load and visualize

In [ ]:
ROBOT_TYPE = RobotType.panda
TIP_LINK = "panda0_hand"
TIMEOUT = 0.5
DATA_DIV = "test"

In [ ]:
rtrain.load_and_visualize(robot_type=ROBOT_TYPE, tip_link=TIP_LINK, data_div=DATA_DIV, timeout=TIMEOUT)